This notebook extracts the lab data and reformat them to have unified denomination. Then extract labels of interest: death and length of stay.

This code relies on [`MIMIC_extract`](https://github.com/MLforHealth/MIMIC_Extract) matching dictionary and the [MIMIC III](https://physionet.org/content/mimiciii/1.4/) data.
To begin, download the data and update the following variable `PATH`.

In [1]:
PATH = '/home/vincent/Desktop/Cambridge/Data/Mimic/' #'/home/vjemj2/rds/hpc-work/data/mimic/'

In [2]:
import numpy as np
import pandas as pd

# Lab data

### Mapping labs variables

In [3]:
mapping = pd.read_csv(PATH + 'itemid_to_variable_map.csv', index_col = 'ITEMID', dtype = {'ITEMID': int})

In [4]:
mapping

,LEVEL2,LEVEL1,ALTERNATIVE,STATUS,STATUS NOTE,MIMIC LABEL,UNITNAME,LINKSTO,COUNT,CATEGORY,CONCEPTID,FLUID,LOINC_CODE,DBSOURCE,Unnamed: 15,PARAM_TYPE,NOTE
ITEMID,,,,,,,,,,,,,,,,,
50861,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALANINE AMINOTRANSFERASE (ALT),NaN,labevents,219475.0,CHEMISTRY,NaN,BLOOD,,NaN,NaN,NaN,NaN
769,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALT,NaN,chartevents,41594.0,Enzymes,NaN,NaN,NaN,carevue,NaN,NaN,NaN
220644,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALT,None,chartevents,37625.0,Labs,NaN,NaN,NaN,metavision,NaN,Numeric,NaN
50862,Albumin,Albumin,NaN,ready,NaN,ALBUMIN,NaN,labevents,146697.0,CHEMISTRY,NaN,BLOOD,1751-7,NaN,NaN,NaN,NaN
772,Albumin,Albumin,NaN,ready,NaN,Albumin (>3.2),NaN,chartevents,31022.0,Chemistry,NaN,NaN,NaN,carevue,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226565,Urine output,Urine output,NaN,ready,NaN,L Nephrostomy,NaN,outputevents,NaN,NaN,NaN,NaN,NaN,metavision,NaN,NaN,NaN
226567,Urine output,Urine output,NaN,ready,NaN,Straight Cath,NaN,outputevents,NaN,NaN,NaN,NaN,NaN,metavision,NaN,NaN,NaN
226584,Urine output,Urine output,NaN,ready,NaN,Ileoconduit,NaN,outputevents,NaN,NaN,NaN,NaN,NaN,metavision,NaN,NaN,NaN


In [5]:
mapping = pd.read_csv(PATH + 'itemid_to_variable_map.csv', index_col = 'ITEMID', dtype = {'ITEMID': int})
mapping = mapping[(mapping['LEVEL2'] != '') &\
                  (mapping['COUNT'] > 0) &\
                  (mapping['STATUS'] == 'ready')
                 ]

In [6]:
mapping.head()

,LEVEL2,LEVEL1,ALTERNATIVE,STATUS,STATUS NOTE,MIMIC LABEL,UNITNAME,LINKSTO,COUNT,CATEGORY,CONCEPTID,FLUID,LOINC_CODE,DBSOURCE,Unnamed: 15,PARAM_TYPE,NOTE
ITEMID,,,,,,,,,,,,,,,,,
50861,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALANINE AMINOTRANSFERASE (ALT),NaN,labevents,219475.0,CHEMISTRY,NaN,BLOOD,,NaN,NaN,NaN,NaN
769,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALT,NaN,chartevents,41594.0,Enzymes,NaN,NaN,NaN,carevue,NaN,NaN,NaN
220644,Alanine aminotransferase,Alanine aminotransferase,ALT,ready,NaN,ALT,None,chartevents,37625.0,Labs,NaN,NaN,NaN,metavision,NaN,Numeric,NaN
50862,Albumin,Albumin,NaN,ready,NaN,ALBUMIN,NaN,labevents,146697.0,CHEMISTRY,NaN,BLOOD,1751-7,NaN,NaN,NaN,NaN
772,Albumin,Albumin,NaN,ready,NaN,Albumin (>3.2),NaN,chartevents,31022.0,Chemistry,NaN,NaN,NaN,carevue,NaN,NaN,NaN


### Extract lab data

In [7]:
labs = pd.read_csv(PATH + 'LABEVENTS.csv', parse_dates = ['CHARTTIME'])

In [8]:
labs

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal
...,...,...,...,...,...,...,...,...,...
27854050,27428435,96443,103219.0,50882,2109-12-30 01:40:00,26,26.00,mEq/L,NaN
27854051,27428436,96443,103219.0,50885,2109-12-30 01:40:00,2.1,2.10,mg/dL,abnormal
27854052,27428437,96443,103219.0,50902,2109-12-30 01:40:00,97,97.00,mEq/L,NaN
27854053,27428438,96443,103219.0,50911,2109-12-30 01:40:00,2,2.00,ng/mL,NaN


In [9]:
# Select data and replace itemid with standard format
labs = labs[labs.ITEMID.isin(mapping.index)][['SUBJECT_ID', 'HADM_ID', 'ITEMID', 'CHARTTIME', 'VALUENUM']]
labs['Lab'] = mapping['LEVEL1'].loc[labs['ITEMID']].values

In [10]:
labs.head()

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUENUM,Lab
0,3,NaN,50820,2101-10-12 16:07:00,7.39,pH
3,3,NaN,50804,2101-10-12 18:17:00,22.00,"CO2 (ETCO2, PCO2, etc.)"
4,3,NaN,50808,2101-10-12 18:17:00,0.93,Calcium ionized
6,3,NaN,50813,2101-10-12 18:17:00,1.80,Lactate
7,3,NaN,50818,2101-10-12 18:17:00,33.00,Partial pressure of carbon dioxide


# Labels

Read the patients' outcome and extract the temporal information for labelling the data

In [11]:
genderAge = pd.read_csv(PATH + 'PATIENTS.csv', usecols = ['SUBJECT_ID', 'GENDER', 'DOB'], parse_dates = ['DOB'])

In [12]:
admissions = pd.read_csv(PATH + 'ADMISSIONS.csv',  
                         usecols = ['SUBJECT_ID', 'HADM_ID', 'ADMISSION_TYPE', 'HOSPITAL_EXPIRE_FLAG',
                                    'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ETHNICITY', 'INSURANCE', 'DIAGNOSIS'],
                         parse_dates = ['ADMITTIME', 'DISCHTIME', 'DEATHTIME'])
admissions = admissions.merge(genderAge, on = 'SUBJECT_ID')

In [13]:
# Focus only on adults - Update 12.10.2021
removed_nan = admissions[['ADMITTIME', 'DOB']].dropna()
admissions['AGE'] = np.nan
admissions['AGE'][removed_nan.index] = [date.days for date in (removed_nan.ADMITTIME.dt.to_pydatetime() - removed_nan.DOB.dt.to_pydatetime())]
admissions.AGE /= 365

admissions = admissions[admissions.AGE > 18]

/tmp/ipykernel_4145/3340935559.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissions['AGE'][removed_nan.index] = [date.days for date in (removed_nan.ADMITTIME.dt.to_pydatetime() - removed_nan.DOB.dt.to_pydatetime())]


In [14]:
# Focus on last visits (as space between visit might change process)
admissions = admissions.loc[admissions['SUBJECT_ID'].drop_duplicates('last').index]

/tmp/ipykernel_4145/2422855185.py:2: FutureWarning: In a future version of pandas all arguments of Series.drop_duplicates will be keyword-only
  admissions = admissions.loc[admissions['SUBJECT_ID'].drop_duplicates('last').index]


In [15]:
# Change times to hours since admission
admissions['LOS'] = admissions['DISCHTIME'] - admissions['ADMITTIME']
admissions['Death'] = admissions['DEATHTIME'] - admissions['ADMITTIME']

In [16]:
# Add temproral information
# Shift of 8 hours to have patients of weekend from 8 am on saturday to 8 am on monday
admissions['Day'] = (admissions['ADMITTIME'] + pd.to_timedelta('8 hours')).dt.weekday

In [17]:
# Create index for easier search
admissions = admissions.set_index('SUBJECT_ID')
admissions.head()

,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INSURANCE,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,DOB,AGE,LOS,Death,Day
SUBJECT_ID,,,,,,,,,,,,,,,
22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,EMERGENCY,Private,WHITE,BENZODIAZEPINE OVERDOSE,0,F,2131-05-07,64.969863,1 days 03:28:00,NaT,5
23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,Medicare,WHITE,BRAIN MASS,0,M,2082-07-17,75.304110,6 days 18:26:00,NaT,2
24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,EMERGENCY,Private,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,M,2100-05-31,39.041096,2 days 20:34:00,NaT,6
25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,EMERGENCY,Private,WHITE,ACUTE CORONARY SYNDROME,0,M,2101-11-21,58.989041,3 days 12:49:00,NaT,6
26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaT,EMERGENCY,Medicare,UNKNOWN/NOT SPECIFIED,V-TACH,0,M,2054-05-04,72.052055,6 days 23:44:00,NaT,0


In [18]:
assert len(admissions.HADM_ID.unique()) == len(admissions), \
    "Different patients have the same HADM_ID, might be a problem for the rest of the code"

# Transformation labs

In [19]:
# Remove unecessary HADM_ID
labs = labs[labs.HADM_ID.isin(admissions.HADM_ID)]

In [20]:
# Change time event to time since admission
labs['Time'] = labs.CHARTTIME.values - admissions.ADMITTIME.loc[labs.SUBJECT_ID].values

In [21]:
labs.head()

,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUENUM,Lab,Time
160,3,145834.0,50868,2101-10-20 16:40:00,17.0,Anion gap,-1 days +21:32:00
161,3,145834.0,50882,2101-10-20 16:40:00,25.0,Bicarbonate,-1 days +21:32:00
162,3,145834.0,50893,2101-10-20 16:40:00,8.2,Calcium (total),-1 days +21:32:00
163,3,145834.0,50902,2101-10-20 16:40:00,99.0,Chloride,-1 days +21:32:00
166,3,145834.0,50912,2101-10-20 16:40:00,3.2,Creatinine,-1 days +21:32:00


# Reformat

Clean reformating of the dataframe for saving

In [22]:
labs.rename(columns = {"SUBJECT_ID": "Patient",
                       "VALUENUM": "Value"}, inplace = True)
labs = labs.reset_index(drop = True)[['Patient', 'Time', 'Lab', 'Value']]
labs.head()

,Patient,Time,Lab,Value
0,3,-1 days +21:32:00,Anion gap,17.0
1,3,-1 days +21:32:00,Bicarbonate,25.0
2,3,-1 days +21:32:00,Calcium (total),8.2
3,3,-1 days +21:32:00,Chloride,99.0
4,3,-1 days +21:32:00,Creatinine,3.2


In [23]:
admissions.rename_axis(index = "Patient", inplace = True)
admissions.head()

,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,INSURANCE,ETHNICITY,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,GENDER,DOB,AGE,LOS,Death,Day
Patient,,,,,,,,,,,,,,,
22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaT,EMERGENCY,Private,WHITE,BENZODIAZEPINE OVERDOSE,0,F,2131-05-07,64.969863,1 days 03:28:00,NaT,5
23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaT,EMERGENCY,Medicare,WHITE,BRAIN MASS,0,M,2082-07-17,75.304110,6 days 18:26:00,NaT,2
24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaT,EMERGENCY,Private,WHITE,INTERIOR MYOCARDIAL INFARCTION,0,M,2100-05-31,39.041096,2 days 20:34:00,NaT,6
25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaT,EMERGENCY,Private,WHITE,ACUTE CORONARY SYNDROME,0,M,2101-11-21,58.989041,3 days 12:49:00,NaT,6
26,197661,2126-05-06 15:16:00,2126-05-13 15:00:00,NaT,EMERGENCY,Medicare,UNKNOWN/NOT SPECIFIED,V-TACH,0,M,2054-05-04,72.052055,6 days 23:44:00,NaT,0


In [24]:
labs.to_csv('data/labs_all.csv', index = False)
admissions.to_csv('data/outcomes_all.csv')

# Selection First days

In [25]:
first_day = pd.to_timedelta('1 day')
admissions = admissions[admissions.LOS >= first_day]
labs = labs[(labs.Time < first_day) &\
              labs.Patient.isin(admissions.index)]

# Cleaning labs

In [26]:
# Remove duplicates: same test multiple time at the same time
labs = labs[~labs.set_index(['Patient', 'Time', 'Lab']).index.duplicated(keep = False)]

In [27]:
# Pivot to hae test as columns
labs = labs.pivot(index = ['Patient', 'Time'], columns = 'Lab')

# Change index to have days
labs.index = labs.index.set_levels(labs.index.levels[1].total_seconds()/(3600.*24), 1)

/tmp/ipykernel_4145/2743300051.py:5: FutureWarning: In a future version of pandas all arguments of MultiIndex.set_levels except for the argument 'levels' will be keyword-only
  labs.index = labs.index.set_levels(labs.index.levels[1].total_seconds()/(3600.*24), 1)


In [28]:
# Keep patients with at least two measures during the 24 hours of admission
# New update for 24 selection => 1.7.2021 
one_measure = labs.index.to_frame(False).groupby('Patient').apply(lambda x: (x < 24).sum()).Time > 1
labs = labs[labs.index.get_level_values(0).isin(one_measure[one_measure].index)]

In [29]:
# Keep labs that at least 5% population has one
# New subselection => 6.7.2021
labs = labs[labs.columns[(labs.groupby('Patient').count()>1).mean() > 0.05]]
labs.head()

Value                             \
Lab               Alanine aminotransferase Albumin Alkaline phosphate   
Patient Time                                                            
3       -0.102778                      NaN     NaN                NaN   
        -0.096528                      NaN     NaN                NaN   
        -0.048611                      NaN     NaN                NaN   
         0.002778                      NaN     NaN                NaN   
         0.004167                      NaN     NaN                NaN   

                                                                             \
Lab               Anion gap Asparate aminotransferase Basophils Bicarbonate   
Patient Time                                                                  
3       -0.102778      17.0                       NaN       0.0        25.0   
        -0.096528       NaN                       NaN       NaN         NaN   
        -0.048611       NaN                       NaN       NaN         NaN   
         0.002778       NaN                       NaN       NaN         NaN   
         0.004167       NaN                       NaN       NaN         NaN   

                                                         \
Lab               Bilirubin (total) Blood urea nitrogen   
Patient Time                                              
3       -0.102778               NaN                53.0   
        -0.096528               NaN                 NaN   
        -0.048611               NaN                 NaN   
         0.002778               NaN                 NaN   
         0.004167               NaN                 NaN   

                                           ...  \
Lab               CO2 (ETCO2, PCO2, etc.)  ...   
Patient Time                               ...   
3       -0.102778                     NaN  ...   
        -0.096528                     NaN  ...   
        -0.048611                     NaN  ...   
         0.002778                    23.0  ...   
         0.004167                    28.0  ...   

                                                              \
Lab               Positive end-expiratory pressure Potassium   
Patient Time                                                   
3       -0.102778                              NaN       5.4   
        -0.096528                              NaN       NaN   
        -0.048611                              NaN       NaN   
         0.002778                              NaN       NaN   
         0.004167                              NaN       NaN   

                                                                       \
Lab               Potassium (whole blood) Red blood cell count Sodium   
Patient Time                                                            
3       -0.102778                     NaN                 3.44  136.0   
        -0.096528                     NaN                  NaN    NaN   
        -0.048611                     NaN                  NaN    NaN   
         0.002778                     4.4                  NaN    NaN   
         0.004167                     4.0                  NaN    NaN   

                                                                         \
Lab               Sodium (whole blood) Tidal Volume Observed Troponin-T   
Patient Time                                                              
3       -0.102778                  NaN                   NaN        NaN   
        -0.096528                  NaN                   NaN        NaN   
        -0.048611                  NaN                   NaN        NaN   
         0.002778                138.0                   NaN        NaN   
         0.004167                153.0                   NaN        NaN   

                                                
Lab               White blood cell count    pH  
Patient Time                                    
3       -0.102778                   12.7   NaN  
        -0.096528                    NaN   NaN  
 

In [30]:
# Keep labs only 24 hours after admission
# Justification: medical process prior to admission might be really different
# # New subselection => 6.7.2021
labs = labs[labs.index.get_level_values('Time') >= 0]
labs.head()

Value                             \
Lab              Alanine aminotransferase Albumin Alkaline phosphate   
Patient Time                                                           
3       0.002778                      NaN     NaN                NaN   
        0.004167                      NaN     NaN                NaN   
        0.012500                      NaN     NaN                NaN   
        0.035417                     25.0     1.8               73.0   
        0.038889                      NaN     NaN                NaN   

                                                                            \
Lab              Anion gap Asparate aminotransferase Basophils Bicarbonate   
Patient Time                                                                 
3       0.002778       NaN                       NaN       NaN         NaN   
        0.004167       NaN                       NaN       NaN         NaN   
        0.012500      23.0                       NaN       NaN        13.0   
        0.035417      22.0                      69.0       NaN        11.0   
        0.038889       NaN                       NaN       NaN         NaN   

                                                        \
Lab              Bilirubin (total) Blood urea nitrogen   
Patient Time                                             
3       0.002778               NaN                 NaN   
        0.004167               NaN                 NaN   
        0.012500               NaN                41.0   
        0.035417               0.8                43.0   
        0.038889               NaN                 NaN   

                                          ...  \
Lab              CO2 (ETCO2, PCO2, etc.)  ...   
Patient Time                              ...   
3       0.002778                    23.0  ...   
        0.004167                    28.0  ...   
        0.012500                     NaN  ...   
        0.035417                     NaN  ...   
        0.038889                    13.0  ...   

                                                             \
Lab              Positive end-expiratory pressure Potassium   
Patient Time                                                  
3       0.002778                              NaN       NaN   
        0.004167                              NaN       NaN   
        0.012500                              NaN       4.0   
        0.035417                              NaN       4.1   
        0.038889                              5.0       NaN   

                                                                      \
Lab              Potassium (whole blood) Red blood cell count Sodium   
Patient Time                                                           
3       0.002778                     4.4                  NaN    NaN   
        0.004167                     4.0                  NaN    NaN   
        0.012500                     NaN                 2.70  143.0   
        0.035417                     NaN                 2.79  140.0   
        0.038889                     3.7                  NaN    NaN   

                                                                        \
Lab              Sodium (whole blood) Tidal Volume Observed Troponin-T   
Patient Time                                                             
3       0.002778                138.0                   NaN        NaN   
        0.004167                153.0                   NaN        NaN   
        0.012500                  NaN                   NaN        NaN   
        0.035417                  NaN                   NaN        NaN   
        0.038889                  NaN                   NaN        NaN   

                                               
Lab              White blood cell count    pH  
Patient Time                                   
3       0.002778                    NaN  7.35  
        0.004167                    NaN  7.59  
        0.012500                   11.3   NaN  
        0.035

In [31]:
# Remove empty lines
labs = labs.dropna(how = 'all')

# Clean outcomes

In [32]:
# Remove patients with no labs
admissions = admissions.loc[labs.index.get_level_values(0).unique()]

In [33]:
admissions['LOS'] = admissions['LOS'].dt.total_seconds() / (24 * 60 * 60)
admissions['Death'] = admissions['Death'].dt.total_seconds() / (24 * 60 * 60)
admissions['Remaining'] = (admissions.LOS.loc[labs.index.get_level_values(0)] - labs.index.get_level_values(1)).groupby('Patient').last()

# Save

Rename columns and save all the data and labels

In [34]:
labs.to_csv('data/labs_first_day.csv')
admissions.to_csv('data/outcomes_first_day.csv')